In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 641s 2s/step - accuracy: 0.5218 - loss: 0.6814 - val_accuracy: 0.7797 - val_loss: 0.4653
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 681s 2s/step - accuracy: 0.8164 - loss: 0.4168 - val_accuracy: 0.8521 - val_loss: 0.3687
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 670s 2s/step - accuracy: 0.8452 - loss: 0.3499 - val_accuracy: 0.8443 - val_loss: 0.3411
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 622s 2s/step - accuracy: 0.8619 - loss: 0.3255 - val_accuracy: 0.8333 - val_loss: 0.3459
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.8592 - loss: 0.3237 - val_accuracy: 0.8443 - val_loss: 0.3354
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 700s 2s/step - accuracy: 0.8660 - loss: 0.3080 - val_accuracy: 0.8510 - val_loss: 0.3228
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.8710 - loss: 0.3040 - val_accuracy: 0.8625 - val_loss: 0.3319
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 622s 2s/step - accuracy: 0.8662 - loss: 0.3093 - val_accu

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
dataset_name = 'imdb'
saved_model_path = './{}_rnn1'.format(dataset_name.replace('/', '_'))

model.save(saved_model_path + '.keras')